<a href="https://colab.research.google.com/github/rohandawar/CarBrandClassificationinDeep-Learning/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link for how to copy kaggle data into google colab:
https://medium.com/analytics-vidhya/how-to-fetch-kaggle-datasets-into-google-colab-ea682569851a

In [2]:
#Mounting the drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/gdrive/My Drive/Kaggle'

In [4]:
#check the present working directory
!pwd

/content


In [6]:
#change to the directory where the file is present
%cd /content/gdrive/My Drive/Kaggle
!pwd

/content/gdrive/My Drive/Kaggle
/content/gdrive/My Drive/Kaggle


In [7]:
#copy and paste the API command from kaggle to collab notebook
!kaggle datasets download -d ritesh2000/car-brand-images-dataset

  0% 0.00/1.06M [00:00<?, ?B/s]
100% 1.06M/1.06M [00:00<00:00, 35.7MB/s]


In [8]:
#Check the content of the diectory
!ls

car-brand-images-dataset.zip  kaggle.json


In [9]:
#unzip the data and remove the ZIP file
!unzip \*.zip && rm *.zip

Archive:  car-brand-images-dataset.zip
  inflating: Test/audi/21.jpg        
  inflating: Test/audi/22.jpg        
  inflating: Test/audi/23.jpg        
  inflating: Test/audi/24.jpg        
  inflating: Test/audi/25.jpg        
  inflating: Test/audi/26.jpg        
  inflating: Test/audi/27.jpg        
  inflating: Test/audi/28.jpg        
  inflating: Test/audi/29.jpg        
  inflating: Test/lamborghini/1.jpg  
  inflating: Test/lamborghini/10.jpg  
  inflating: Test/lamborghini/11.jpg  
  inflating: Test/lamborghini/12.jpg  
  inflating: Test/lamborghini/13.jpg  
  inflating: Test/lamborghini/14.jpg  
  inflating: Test/lamborghini/15.jpg  
  inflating: Test/lamborghini/16.jpg  
  inflating: Test/lamborghini/17.jpg  
  inflating: Test/lamborghini/18.jpg  
  inflating: Test/lamborghini/19.jpg  
  inflating: Test/lamborghini/2.jpg  
  inflating: Test/lamborghini/20.jog.jpg  
  inflating: Test/lamborghini/21.jpg  
  inflating: Test/lamborghini/22.jpg  
  inflating: Test/lamborghini/23

In [10]:
# Importing the important Libraries
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50

#import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from tensorflow.keras.models import Sequential

import numpy as np
import matplotlib.pyplot as plt

from glob import glob

In [12]:
#resizing all the images to 224, 224
IMAGE_SIZE = [224, 224]

train_path = 'Train'
test_path = 'Test'